<a href="https://colab.research.google.com/github/akiFQC/colaboratry_DNN_API_collection/blob/master/image_classifier_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask web application of image classifier by pretrained VGG16

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-08-04 13:57:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.203.102.189, 52.21.103.149, 52.86.186.182, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.203.102.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  38.8MB/s    in 0.3s    

2019-08-04 13:57:33 (38.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
from flask import Flask, render_template, send_from_directory, request, redirect, url_for

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import Variable
from chainer.backends import cuda

import numpy as np
import pandas as pd
import cupy as cp

import os, sys
import pprint
import time
import urllib.error
import urllib.request
from PIL import Image
import io
import random, string



In [0]:
def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file:
            data = web_file.read()
            with open(dst_path, mode='wb') as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print(e)

In [0]:
# カテゴリーのリストを取得
download_file( "https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt", "category.txt")

category={}

def extract_line(line):
    line =line.replace("{", "")
    line=line.replace("\}", "")
    line=line.replace("\'", "")
    line=line.replace("\n", "")
    idx, name = line.split(":")
    return idx, name[:-1]

with open("category.txt", "r") as f :
    lili= f.readlines() 


for li in lili:
    idx, name = extract_line(li)
    category[int(idx)]=name

In [0]:
def randomname(n):
    randlst = [random.choice(string.ascii_letters + string.digits) for i in range(n)]
    return ''.join(randlst)

In [0]:
html = """

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
</head>
<body>

<form action="/post" method="post" enctype="multipart/form-data">
    <p><input type="file" name="uploadFile"/></p>
    <p><input type="submit" value="send"/></p>
</form>

{% if ulr_Image %}

    <p>Image</p>
    <p><img src="{{ ulr_Image }}"></p>
    <p>Result</p>
    <p>{{ result_class }}</p>
    <p>{{ result_score }}</p>

{% endif %}

</body>
</html>

"""

In [0]:
#Saving html file
if not os.path.isdir( "templates" ):
    os.makedirs( "templates" )
with open("templates/index.html", mode='w') as f:
    f.write(html)
    
#Making tmp folder to use temporarily stored
if not os.path.isdir( "tmp" ):
    os.makedirs( "tmp" )

In [8]:
model =  L.VGG16Layers()
model.to_gpu(0)
model.disable_update()

Now loading caffemodel (usually it may take few minutes)


In [9]:
# テスト用
url = "https://upload.wikimedia.org/wikipedia/commons/5/5c/Gro%C3%9Fer_Schweizer_Sennenhund.jpg"

data_path =  "tmp/" + url.split("/")[-1]
print(data_path)
download_file( url, data_path)

def prediction_from_path(path):
    img = Image.open(path)
    X_img =  L.model.vision.vgg.prepare(img)

    X_img = X_img[np.newaxis]

    x_cp = cp.asarray(X_img)
    x_cp = cuda.to_gpu(x_cp)

    result = model(x_cp)

    idx = int(F.argmax((result['prob']), axis=1).data)
    label = category[idx]
    prob = float(F.max(result['prob'], axis=1).data)
    return label, prob
    
print(prediction_from_path(data_path))

    

tmp/Gro%C3%9Fer_Schweizer_Sennenhund.jpg
(' Greater Swiss Mountain dog', 0.8891767263412476)


In [0]:
app = Flask(__name__)

@app.route("/")
def index():
    return render_template( "index.html" )

@app.route('/post', methods=['POST'])
def post():
    try:
        uploadFile = request.files['uploadFile']
    except:
        return redirect(url_for('index'))
    
    file_name= fileName = randomname(24) + ".jpg"

    uploadFile.save("tmp/" + file_name)

    img = Image.open("tmp/" + file_name)

    url_image = "tmp/" + file_name
    label, prob = prediction_from_path(url_image)
    
    
    print('predicted_id:', label, ", prob:", prob)
    
    result_class = "label: " + label
    result_score = "Score: " + str(prob)

    return render_template( "index.html",
                                        ulr_Image = url_image,
                                        result_class = result_class,
                                        result_score = result_score)
    
@app.route('/tmp/<filename>')
def uploaded_file(filename):
    return send_from_directory("./tmp", filename)
    


In [11]:
!cd ~
! ls
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


if __name__ == '__main__':
    app.run(port=6006)

category.txt  ngrok  ngrok-stable-linux-amd64.zip  sample_data	templates  tmp
http://45777d2d.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6006/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Aug/2019 13:59:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2019 13:59:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Aug/2019 13:59:31] "POST /post HTTP/1.1" 200 -


predicted_id:  Greater Swiss Mountain dog , prob: 0.9902670979499817


127.0.0.1 - - [04/Aug/2019 13:59:31] "GET /tmp/elCL3cGeLzWpOkptUDMoAvJo.jpg HTTP/1.1" 200 -
